## Using Jupyter Notebook and Pandas to Extract Data From a MySQL Database.

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import pymysql
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [19]:
## Conneting to the database.
conn = pymysql.connect(host = "localhost",
                               user = "root",
                               passwd = "Mominulhasan1",
                               db = "Data_Analysis")
conn

In [20]:
## Checking the table names from Data_Analysis database.

Data_Analysis_tables = pd.read_sql_query("SHOW TABLES FROM Data_Analysis", conn)
Data_Analysis_tables

,Tables_in_data_analysis
0,general_ledger_accounts
1,invoice_line_items
2,invoices
3,terms
4,vendors


In [21]:
## Investigating every columns in each table in the data_analysis database.

tables = Data_Analysis_tables["Tables_in_data_analysis"]
for table_name in tables:
    output = pd.read_sql_query("DESCRIBE {}" .format(table_name), conn)
    print(table_name)
    print(output, "\n")

general_ledger_accounts
                 Field         Type Null  Key Default Extra
0       account_number          int   NO  PRI    None      
1  account_description  varchar(50)  YES  UNI    None       

invoice_line_items
                   Field          Type Null  Key Default Extra
0             invoice_id           int   NO  PRI    None      
1       invoice_sequence           int   NO  PRI    None      
2         account_number           int   NO  MUL    None      
3       line_item_amount  decimal(9,2)   NO         None      
4  line_item_description  varchar(100)   NO         None       

invoices
              Field          Type Null  Key Default           Extra
0        invoice_id           int   NO  PRI    None  auto_increment
1         vendor_id           int   NO  MUL    None                
2    invoice_number   varchar(50)   NO         None                
3      invoice_date          date   NO  MUL    None                
4     invoice_total  decimal(9,2)   NO        

In [6]:
## Fetching the following columns from the "invoices" table:
##
##  - Invoice Number        (The "invoice_number" column)
##  - Invoice Total         (The "invoice_total" column)
##  - Payment Credit Total  ("payment_total" + "credit_total")
##  - Balance Due           ("invoice_total" - "payment_total" - "credit_total")
##
## Only returning invoices that have a balance due that is greater than $50.
## Sorting the result set by balance due in descending order and return only the rows with the 5 largest balance due.


df_invoice = pd.read_sql_query("SELECT\
                                invoice_number AS Invoice_Number,\
                                invoice_total AS Invoice_Total,\
                                (payment_total + credit_total) AS Payment_Credit_Total,\
                                (invoice_total - payment_total - credit_total) AS Balance_Due\
                                FROM\
                                invoices\
                                WHERE\
                                (invoice_total - payment_total - credit_total) > 50\
                                ORDER BY (invoice_total - payment_total - credit_total) DESC\
                                LIMIT 5;",
                                conn)

df_invoice

,Invoice_Number,Invoice_Total,Payment_Credit_Total,Balance_Due
0,P-0608,20551.18,1200.0,19351.18
1,0-2436,10976.06,0.0,10976.06
2,31361833,579.42,0.0,579.42
3,9982771,503.20,0.0,503.20
4,547480102,224.00,0.0,224.00


In [7]:
## Returning the identities of the contact person from the "vendors" table.
## Returning only the contacts whose last name begins with the letter A, B, C or E.
## Sorting the result set by last name and then first name in ascending order.

df_vendors = pd.read_sql_query("SELECT vendor_contact_last_name AS Last_Name,\
                                vendor_contact_first_name AS First_Name\
                                FROM\
                                vendors\
                                WHERE\
                                vendor_contact_last_name LIKE ('a%')\
                                OR vendor_contact_last_name LIKE ('b%')\
                                OR vendor_contact_last_name LIKE ('c%')\
                                OR vendor_contact_last_name LIKE ('e%')\
                                ORDER BY vendor_contact_last_name,\
                                vendor_contact_first_name\
                                LIMIT 10;",
                               conn)

df_vendors

,Last_Name,First_Name
0,Aaronsen,Thom
1,Aileen,Joan
2,Alberto,Francesco
3,Alexis,Alexandro
4,Alondra,Zev
5,Angelica,Nashalie
6,Antavius,Troy
7,Anthoni,Kaitlyn
8,Anum,Trisha
9,Aranovitch,Robert


In [8]:
## Two tables data will be used for this analysis such as "invoices" and "vendors"

## Identifying for each vendor, the invoices with a non-zero balance due.
##
## Returning the following columns in the result set:
##
##  - Vendor Name     (The "vendor_name" column from the "vendors" table)
##  - Invoice Number  (The "invoice_number" column from the "invoices" table)
##  - Invoice Date    (The "invoice_date" column from the "invoices" table)
##  - Balance Due     ("invoice_total" - "payment_total" - "credit_total")
##
## Sorting the result set by "vendor_name" in ascending order.


df_vendor_invoice = pd.read_sql_query("SELECT t2.vendor_name Vendor_Name, invoice_number Invoice_Number,\
                                        invoice_date Invoice_Date,\
                                        (invoice_total - payment_total - credit_total) Balance_Due\
                                        FROM\
                                        invoices AS t1 \
                                        INNER JOIN \
                                        vendors AS t2 ON t1.vendor_id = t2.vendor_id\
                                        WHERE\
                                        (invoice_total - payment_total - credit_total) > 0 \
                                        ORDER BY vendor_name ASC;",
                                     conn)

df_vendor_invoice

,Vendor_Name,Invoice_Number,Invoice_Date,Balance_Due
0,Blue Cross,547480102,2014-08-01,224.00
1,"Cardinal Business Media, Inc.",134116,2014-07-28,90.36
2,Data Reproductions Corp,39104,2014-07-10,85.31
3,Federal Express Corporation,963253264,2014-07-18,52.25
4,Federal Express Corporation,263253268,2014-07-21,59.97
5,Federal Express Corporation,263253270,2014-07-22,67.92
6,Federal Express Corporation,263253273,2014-07-22,30.75
7,Ford Motor Credit Company,9982771,2014-07-24,503.20
8,Ingram,31361833,2014-07-21,579.42
9,Malloy Lithographing Inc,P-0608,2014-07-23,19351.18


In [9]:
## Returning one row for each vendor, which contains the following values:
##
##  - Vendor Name  (The "vendor_name" column from the "vendors" table)
##  - The number of invoices (from the "invoices" table) for the vendor
##  - The sum of "invoice_total" (from the "invoices" table) for the vendor
##
## Sorting the result set such that the vendor with the most invoices appears first.


df_count_vendor = pd.read_sql_query("SELECT DISTINCT \
                                        vendor_name Vendor_Name,\
                                        count(invoice_number) The_Number_of_Invoices,\
                                        sum(invoice_total) Total_Invoice\
                                    FROM\
                                        invoices AS t1\
                                            INNER JOIN\
                                        vendors AS t2 ON t1.vendor_id = t2.vendor_id\
                                    GROUP BY vendor_name\
                                    ORDER BY count(invoice_number) DESC \
                                    LIMIT 20;",
                                    conn)
df_count_vendor

,Vendor_Name,The_Number_of_Invoices,Total_Invoice
0,Federal Express Corporation,47,4378.02
1,United Parcel Service,9,23177.96
2,Zylka Design,8,6940.25
3,Pacific Bell,6,171.01
4,Malloy Lithographing Inc,5,119892.41
5,"Roadway Package System, Inc",4,43.67
6,Blue Cross,3,564.00
7,Compuserve,2,19.90
8,IBM,2,1200.12
9,Data Reproductions Corp,2,21927.31


In [10]:
## Identifying the accounts (from the "general_ledger_accounts" table),
## which do not match any invoice line items in the "invoice_line_items" table.
##
## Returning the following two columns in the result set:
##
##  - "account_number" (from the "general_ledger_accounts" table)
##  - "account_description" (from the "general_ledger_accounts" table)
##
## Sorting the result set by account number in ascending order.

## NOTE: THREE different methods have been used for this analysis such as LEFT JOIN, subquery NOT IN and NOT EXISTS.
## Writing one query for each method.

## Method 1 with LEFT JOIN operator:

df_method1 = pd.read_sql_query("SELECT DISTINCT T1.account_number Account_Number,T1.account_description Account_Description\
                            FROM general_ledger_accounts AS T1\
                                LEFT JOIN\
                                invoice_line_items AS T2 ON T1.account_number = T2.account_number\
                            WHERE invoice_id IS NULL\
                            ORDER BY T1.account_number ASC",
                            conn)
df_method1

,Account_Number,Account_Description
0,100,Cash
1,110,Accounts Receivable
2,120,Book Inventory
3,162,Capitalized Lease
4,167,Software
5,181,Book Development
6,200,Accounts Payable
7,205,Royalties Payable
8,221,401K Employee Contributions
9,230,Sales Taxes Payable


In [11]:
## ## Method 2 with a subquery and NOT IN operator :

df_method2 = pd.read_sql_query("SELECT DISTINCT \
                                    account_number AS Account_Number,\
                                    account_description AS Account_Description\
                                FROM general_ledger_accounts \
                                WHERE account_number NOT IN (SELECT DISTINCT account_number \
                                                             FROM invoice_line_items)\
                                ORDER BY account_number",
                                conn)
df_method2

,Account_Number,Account_Description
0,100,Cash
1,110,Accounts Receivable
2,120,Book Inventory
3,162,Capitalized Lease
4,167,Software
5,181,Book Development
6,200,Accounts Payable
7,205,Royalties Payable
8,221,401K Employee Contributions
9,230,Sales Taxes Payable


In [12]:
## Method 3 with a subquery and NOT EXISTS operator:

df_method3 = pd.read_sql_query("SELECT DISTINCT \
                                    account_number AS Account_Number,\
                                    account_description AS Account_Description\
                                FROM general_ledger_accounts\
                                WHERE NOT EXISTS(SELECT DISTINCT * \
                                                FROM invoice_line_items \
                                                WHERE invoice_line_items.account_number = general_ledger_accounts.account_number)\
                                ORDER BY account_number ASC",
                               conn)
df_method3                                

,Account_Number,Account_Description
0,100,Cash
1,110,Accounts Receivable
2,120,Book Inventory
3,162,Capitalized Lease
4,167,Software
5,181,Book Development
6,200,Accounts Payable
7,205,Royalties Payable
8,221,401K Employee Contributions
9,230,Sales Taxes Payable


In [13]:
## Returning one row per vendor, which represents the vendor's oldest invoice (the one with the earliest date).
##
## Each row returned should include the following values:
##
##  - "vendor_name"
##  - "invoice_number"
##  - "invoice_date"
##  - "invoice_total"
##
## Sorting the result set by "vendor_name" in ascending order.

## Assigning sql queries to mn

mn = "SELECT DISTINCT vendor_name,invoice_number,invoice_total,invoice_date Vendor_Oldest_Invoice\
    FROM invoices AS t1\
        INNER JOIN\
    vendors AS t2 ON t1.vendor_id = t2.vendor_id\
    WHERE\
        invoice_date = (SELECT MIN(invoice_date)\
                        FROM invoices\
                        WHERE\
                        vendor_id = t1.vendor_id) \
    ORDER BY vendor_name ASC"

In [14]:
## Reading sql query through pandas
vendors_oldest_invoice = pd.read_sql_query(mn, conn)
vendors_oldest_invoice

,vendor_name,invoice_number,invoice_total,Vendor_Oldest_Invoice
0,Abbey Office Furnishings,203339-13,17.50,2014-07-05
1,Bertelsmann Industry Svcs. Inc,509786,6940.25,2014-06-18
2,Blue Cross,547481328,224.00,2014-06-03
3,Cahners Publishing Company,587056,2184.50,2014-06-30
4,"Cardinal Business Media, Inc.",133560,175.00,2014-06-22
5,Coffee Break Service,109596,41.80,2014-06-24
6,Compuserve,21-4748363,9.95,2014-05-03
7,Computerworld,367447,2433.00,2014-06-11
8,Data Reproductions Corp,40318,21842.00,2014-06-01
9,Dean Witter Reynolds,75C-90227,1367.50,2014-06-11


In [15]:
## Identifying the invoices, whose payment total is greater than the average payment total
## of all the invoices with a non-zero payment total.
##
## Returning the "invoice_number" and "invoice_total" for each invoice.
## Sorting the result set by "invoice_total" in descending order.

greater_than_avg_payment_total = pd.read_sql_query("SELECT DISTINCT invoice_number, invoice_total FROM invoices \
                                                    WHERE payment_total > 0 \
                                                        AND payment_total > (SELECT AVG(payment_total) \
                                                                            FROM invoices) \
                                                    ORDER BY invoice_total DESC",
                                                   conn)
greater_than_avg_payment_total

,invoice_number,invoice_total
0,0-2058,37966.19
1,P-0259,26881.40
2,0-2060,23517.58
3,40318,21842.00
4,P02-3772,7125.34
5,509786,6940.25
6,10843,4901.26
7,989319-457,3813.33
8,989319-447,3689.99
9,989319-437,2765.36


In [16]:
## Returning one row for each general ledger account, which contains the following values:
##
##  - Account Number (The "account_number" column from the "general_ledger_accounts" table)
##  - Account Description  (The "account_description" column from the "general_ledger_accounts" table)
##  - The number of items in the "invoice_line_items" table that are related to the account
##  - The sum of "line_item_amount" of the account
##
## Returning only those accounts, whose sum of line item amount is great than $5,000.
## The result set should also be sorted by the sum of line item amount in descending order.

## Assigning the queries to ledger_account

ledger_account = "SELECT t1.account_number,t1.account_description,count(*) AS Number_of_Line_Items,\
                    sum(line_item_amount) AS Sum_of_Line_Amount\
                FROM general_ledger_accounts AS t1\
                        INNER JOIN\
                    invoice_line_items AS t2 ON t1.account_number = t2.account_number\
                GROUP BY t1.account_number, t1.account_description HAVING sum(line_item_amount) > 5000 \
                ORDER BY sum(line_item_amount) DESC"


In [18]:
## Reading the query through pandas

ledger_account_count = pd.read_sql_query(ledger_account,conn)
ledger_account_count

,account_number,account_description,Number_of_Line_Items,Sum_of_Line_Amount
0,400,Book Printing Costs,8,148759.97
1,553,Freight,60,27599.65
2,589,Outside Services,3,13394.10
3,403,Book Production Costs,8,6175.12
4,572,"Books, Dues, and Subscriptions",6,5207.32
